In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import heartbeat as hb


person_1_mild = pd.read_excel('Manoj_Thesis/RawData/P_12_S_0.xlsx')
person_1_medium = pd.read_excel('Manoj_Thesis/RawData/P_12_S_1.xlsx')
person_1_high = pd.read_excel('Manoj_Thesis/RawData/P_12_S_2.xlsx')
person_1_high=np.asarray(person_1_high)
measure=hb.process(person_1_medium,100)
print(measure)

ValueError: negative dimensions are not allowed

In [ ]:
#f,ax = plt.subplots(3)

#plt.plot(person_1_mild[0:1000],color = 'g')
#plt.plot(person_1_medium[7000:8000],label = 'medium_stress',color = 'm')
plt.plot(person_1_high[1000:2000],color = 'g')
plt.xlabel('time_intervals')
plt.ylabel('frequency')
plt.legend(fontsize = 'medium')
plt.show()



In [7]:
hrw = 0.75
import math
fs = 100
mov_avg = person_1_high.rolling(int(hrw*fs)).mean() #Calculate moving average
#Impute where moving average function returns NaN, which is the beginning of the signal where x hrw
avg_hr = (np.mean(person_1_high))
mov_avg = [avg_hr if math.isnan(x) else x for x in mov_avg]
mov_avg = [x*1.2 for x in mov_avg] #For now we raise the average by 20% to prevent the secondary heart contraction from interfering, in part 2 we will do this dynamically
print(mov_avg)
person_1_high['hart_rollingmean'] = mov_avg #Append the moving average to the dataframe
#Mark regions of interest
window = []
peaklist = []
listpos = 0 #We use a counter to move over the different data columns
for datapoint in person_1_high:
    rollingmean = person_1_high.hart_rollingmean[listpos] #Get local mean
    if (datapoint < rollingmean) and (len(window) < 1): #If no detectable R-complex activity -> do nothing
        listpos += 1
    elif (datapoint > rollingmean): #If signal comes above local mean, mark ROI
        window.append(datapoint)
        listpos += 1
    else: #If signal drops below local mean -> determine highest point
        maximum = max(window)
        beatposition = listpos - len(window) + (window.index(max(window))) #Notate the position of the point on the X-axis
        peaklist.append(beatposition) #Add detected peak to list
        window = [] #Clear marked ROI
        listpos += 1
ybeat = [person_1_high[x] for x in peaklist] #Get the y-value of all peaks for plotting purposes
plt.title("Detected peaks in signal")
plt.xlim(0,2500)
plt.plot(person_1_high, alpha=0.5, color='blue') #Plot semi-transparent HR
plt.plot(mov_avg, color ='green') #Plot moving average
plt.scatter(peaklist, ybeat, color='red') #Plot detected peaks
plt.show()

[0.0]


ValueError: Length of values does not match length of index